## Unconstrained Optimization - Optimization Algorithms

In [1]:
import numpy as np
import numdifftools as nd
import copy
import time

In [ ]:
! pip install numdifftools

## numdifftools 사용 방법

In [2]:
fun = lambda x : 4*x[1]**3+x[0]**2-12*x[1]**2-36*x[1]+2

In [3]:
H = nd.Hessian(fun)

In [4]:
H(np.array([1, 1]))

array([[2., 0.],
       [0., 0.]])

In [5]:
g = nd.Gradient(fun)

In [6]:
g([1,1])

array([  2., -48.])

## Steepest Gradient

In [7]:
def gradient_descent(fun, x0, step_length, max_iter=1000, epsilon=1e-3):
    
    fun = fun
    x = copy.copy(x0)
    
    grad = nd.Gradient(fun)
    
    count = 0
    for i in range(max_iter):
        gradx = grad(x)
        x -= step_length*gradx
        count += 1
        if np.linalg.norm(gradx)<epsilon:
            break
    return x, count

## Newton's Method

In [8]:
def newton_method(fun, x0, max_iter=1000, epsilon=1e-3):
    
    fun = fun
    x = copy.copy(x0)
    
    grad = nd.Gradient(fun)
    hess = nd.Hessian(fun)
    
    count = 0
    for i in range(max_iter):
        gradx = grad(x)
        hessx = hess(x)
        hessx = np.linalg.inv(hessx)
        x -= hessx.dot(gradx)
        count += 1
        if np.linalg.norm(gradx)<epsilon:
            break
    return x, count

## Quasi-Newton (BFGS) Method

In [9]:
def BFGS(fun, x0, step_length, max_iter=1000, epsilon=1e-3):
    
    fun = fun
    x = copy.copy(x0)
    H = np.eye(len(x))
    grad = nd.Gradient(fun)
    
    count = 0
    for i in range(max_iter):
        gradx = grad(x)
        x_new = x-step_length*H.dot(gradx)
        count+=1
        if np.linalg.norm(gradx)<epsilon:
            break
        
        new_gradx = grad(x_new)
        s = x_new-x
        y = new_gradx-gradx
        rho = 1/s.dot(y)
        
        H_new = (np.eye(len(x))-rho*np.dot(s.reshape(-1,1),y.reshape(1,-1))).dot(H)
        H_new = H_new.dot(np.eye(len(x))-rho*np.dot(y.reshape(-1,1),s.reshape(1,-1)))
        H_new = H_new+rho*np.dot(s.reshape(-1,1), s.reshape(1,-1))
        
        x = x_new
        H = H_new
        
    return x, count        

In [11]:
for step_length in range(10):
    print(gradient_descent(fun, np.array([1., 2.]), 0.01*(1+step_length)))

(array([4.82454778e-04, 3.00000000e+00]), 378)
(array([4.64506517e-04, 3.00000000e+00]), 188)
(array([4.65429436e-04, 3.00000000e+00]), 124)
(array([4.51376850e-05, 2.99998237e+00]), 120)
(array([3.58218875e-13, 1.56112507e+00]), 1000)
(array([1.28984400e-14, 3.61984044e+00]), 1000)
(array([ 4.04802018e-01, -1.12160100e+31]), 9)
(array([ 4.18211936e-01, -2.95525227e+26]), 8)
(array([ 3.70559624e-01, -1.80954158e+17]), 7)
(array([ 4.09599983e-01, -3.79398415e+20]), 7)


In [12]:
for step_length in range(20):
    print(BFGS(fun, np.array([1.,2.]), 0.01*(1+step_length)))

(array([4.23599087e-05, 2.99997670e+00]), 1000)
(array([7.37418006e-05, 2.99997981e+00]), 470)
(array([2.09734529e-04, 3.00001853e+00]), 277)
(array([3.70717849e-05, 3.00002014e+00]), 249)
(array([1.81133596e-05, 3.00001993e+00]), 212)
(array([1.14672610e-05, 3.00002032e+00]), 183)
(array([7.92261892e-06, 3.00002037e+00]), 161)
(array([5.67831669e-06, 3.00001995e+00]), 144)
(array([4.35909051e-06, 3.00002012e+00]), 130)
(array([3.26325648e-06, 3.00001926e+00]), 119)
(array([2.73241662e-06, 3.00002021e+00]), 109)
(array([2.18479470e-06, 3.00001995e+00]), 101)
(array([1.79628528e-06, 3.00002001e+00]), 94)
(array([1.47060108e-06, 3.00001981e+00]), 88)
(array([1.16007700e-06, 3.00001877e+00]), 83)
(array([1.01494744e-06, 3.00001963e+00]), 78)
(array([8.18534496e-07, 3.00001886e+00]), 74)
(array([7.16543745e-07, 3.00001963e+00]), 70)
(array([5.51948883e-07, 3.00001797e+00]), 67)
(array([4.50865415e-07, 3.00001746e+00]), 64)


In [13]:
newton_method(fun, np.array([1.,2.]))

(array([1.99840144e-15, 3.00000000e+00]), 5)

In [14]:
BFGS(fun, np.array([1.,2.]), 1)

(array([6.53343067e-06, 3.00000021e+00]), 9)

In [15]:
start = time.time()
newton_method(fun, np.array([1.,2.]))
end = time.time()
end-start

0.04897451400756836

In [16]:
start = time.time()
BFGS(fun, np.array([1.,2.]), 1)
end = time.time()
end-start

0.050971269607543945